# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Moduł 5: Statystyczne tłumaczenie maszynowe

## Zadanie 1
Zadanie polega na zaimplementowaniu algorytmu Expectation-Maximization w modelu IBM Model 1 do przypasowywania słów. Będzie to fragment modelu, który tłumaczyć będzie z hiszpańskiego na angielski. 

UWAGA: Specjalny token "NULL" pomijamy w implementacji.

Dany jest mini-korpus równoległy angielsko-hiszpański
- "green house" "casa verde"
- "the house" "la casa"
- "the green house" "la casa verde"


In [1]:
import itertools
english = [["green","house"], ["the","house"], ["the", "green", "house"]]
spanish = [["casa", "verde"], ["la", "casa"], ["la", "casa", "verde"]]

W dalszych funkcjach przydatne może być wyznaczenie słownika czyli zbioru słów z korpusu dla danego języka.

In [2]:
def get_vocabulary(corpus):
    """
    Funkcja zwracająca listę unikalnych słów z korpusu podanego w formacie zmiennej english i spanish
    """
    return set(word for phrase in corpus for word in phrase)

In [3]:
from nose.tools import assert_set_equal
assert_set_equal(set(get_vocabulary(english)), set(["the", "green", "house"]))

Zainicjalizuj rozkład prawdopodobieństwa tłumaczenia słów rozkładem jednorodnym. Ponieważ zależy nam na prostocie implementacji (a nie efektywności) możemy to prawdopodobieństwo zaimplementować jako zwykły słownik, który będzie przyjmował na wejściu krotkę dwóch słów. Słownik nazwij `translation_prob` z kluczami (słowo_es, słowo_en).

In [4]:
def initalize_translation_prob(target_corpus, source_corpus):
    target_vocab = get_vocabulary(target_corpus)
    source_vocab = get_vocabulary(source_corpus)
    uniform_probability = 1 / len(target_vocab)
    return {pair: uniform_probability for pair in itertools.product(source_vocab, target_vocab)}
translation_prob = initalize_translation_prob(english, spanish)

Wypisz zaincjalizowany słownik, żeby upewnić się że wynik jest prawidłowy.

In [5]:
translation_prob

{('la', 'house'): 0.3333333333333333,
 ('la', 'green'): 0.3333333333333333,
 ('la', 'the'): 0.3333333333333333,
 ('verde', 'house'): 0.3333333333333333,
 ('verde', 'green'): 0.3333333333333333,
 ('verde', 'the'): 0.3333333333333333,
 ('casa', 'house'): 0.3333333333333333,
 ('casa', 'green'): 0.3333333333333333,
 ('casa', 'the'): 0.3333333333333333}

Zaimplementuj pierwszy krok algorytmu EM. Wyznacz wartości oczekiwane zmiennych przypisania słowa we wszystkich zdaniach w korpusie (oznaczane na wykładzie jako `a`).

In [6]:
import collections

def normalize_dict(distribution, key_selector):
    """
    Funkcja przyjmuje jako argumenty słownik i inną funkcję,
    która zwraca fragment klucza tego słownika.
    W wyniku zwraca słownik, w którym wszystkie wartości są znormalizowane
    po sumie dla fragmentu klucza zwracanego przez key_selector.
    
    >>> normalize_dict({(0, 1): 3, (0, 2): 2, (1, 1): 4, (1, 2): 4}, lambda key: key[0])
    {(0, 1): 0.6, (0, 2): 0.4, (1, 1): 0.5, (1, 2): 0.5}
    """
    totals = collections.defaultdict(float)
    for key, value in distribution.items():
        totals[key_selector(key)] += value
    return { key: value / totals[key_selector(key)] for key, value in distribution.items()}

def calculate_expectation(target_corpus, source_corpus, translation_prob):
    """
    Procedura wykonująca krok "E" algorytmu EM
    Wynikiem powinny być wartości oczekiwane dla zmiennej przypisań słów w zdaniach 
    (reprezentacja dowolna, nie weryfikowana przez sprawdzarkę)
    """
    expectation = {}
    for k, (target_sentence, source_sentence) in enumerate(zip(target_corpus, source_corpus)):
        sentence_total = collections.defaultdict(float)
        for target_word, source_word in itertools.product(target_sentence, source_sentence):
            word_pair_prob = translation_prob[source_word, target_word]
            expectation[k, target_word, source_word] = word_pair_prob
            sentence_total[target_word] += word_pair_prob
    return normalize_dict(expectation, lambda key: key[0:2])

assigment_expected_values = calculate_expectation(english, spanish, translation_prob)

Wypisz wartości oczekiwane zmiennych przypisań, aby zobaczyć jak wyglądają. Powinny one również prezentować całkowity brak wiedzy o przypisaniach (rozkłady jednorodne).

In [7]:
assigment_expected_values

{(0, 'green', 'casa'): 0.5,
 (0, 'green', 'verde'): 0.5,
 (0, 'house', 'casa'): 0.5,
 (0, 'house', 'verde'): 0.5,
 (1, 'the', 'la'): 0.5,
 (1, 'the', 'casa'): 0.5,
 (1, 'house', 'la'): 0.5,
 (1, 'house', 'casa'): 0.5,
 (2, 'the', 'la'): 0.3333333333333333,
 (2, 'the', 'casa'): 0.3333333333333333,
 (2, 'the', 'verde'): 0.3333333333333333,
 (2, 'green', 'la'): 0.3333333333333333,
 (2, 'green', 'casa'): 0.3333333333333333,
 (2, 'green', 'verde'): 0.3333333333333333,
 (2, 'house', 'la'): 0.3333333333333333,
 (2, 'house', 'casa'): 0.3333333333333333,
 (2, 'house', 'verde'): 0.3333333333333333}

Zaimplementuj drugi krok algorytmu EM. Wyznacz nowe `translation_prob` na podstawie oczekiwanych wartości zmiennych przypisań.

In [8]:
def calculate_maximization(target_corpus, source_corpus, assignment_expected_values):
    new_translation_prob = collections.defaultdict(float)
    word_total = collections.defaultdict(float)
    for (k, target_word, source_word), prob in assignment_expected_values.items():
        new_translation_prob[source_word, target_word] += prob
        word_total[target_word] += prob
    return normalize_dict(new_translation_prob, lambda key: key[1])

translation_prob = calculate_maximization(english, spanish, assigment_expected_values)

In [9]:
from nose.tools import assert_almost_equal
assert_almost_equal(translation_prob[('casa', 'house')], 4/9.)
assert_almost_equal(translation_prob[('la', 'house')], 5/18.)

Wywołaj w pętli 10 kroków algorytmu EM i zaobserwuj jak zmieniają się prawdopodobieństwa dla tłumacznienia "house".

In [10]:
for i in range(10):
    assigment_expected_values = calculate_expectation(english, spanish, translation_prob)
    translation_prob = calculate_maximization(english, spanish, assigment_expected_values)
    print([(i,j) for i,j in translation_prob.items() if i[1] == "house"])
    print("---")


[(('casa', 'house'), 0.5584045584045584), (('verde', 'house'), 0.22079772079772075), (('la', 'house'), 0.22079772079772075)]
---
[(('casa', 'house'), 0.6638923177619095), (('verde', 'house'), 0.16805384111904523), (('la', 'house'), 0.16805384111904523)]
---
[(('casa', 'house'), 0.7532968646774506), (('verde', 'house'), 0.12335156766127475), (('la', 'house'), 0.12335156766127475)]
---
[(('casa', 'house'), 0.8239601969358897), (('verde', 'house'), 0.08801990153205519), (('la', 'house'), 0.08801990153205519)]
---
[(('casa', 'house'), 0.8769766282684491), (('verde', 'house'), 0.06151168586577549), (('la', 'house'), 0.06151168586577549)]
---
[(('casa', 'house'), 0.915296630096382), (('verde', 'house'), 0.042351684951809), (('la', 'house'), 0.042351684951809)]
---
[(('casa', 'house'), 0.9422824270785402), (('verde', 'house'), 0.02885878646072994), (('la', 'house'), 0.02885878646072994)]
---
[(('casa', 'house'), 0.9609498992371662), (('verde', 'house'), 0.019525050381416928), (('la', 'house')

Wywołaj algorytm EM na poniższym korpusie.

In [ ]:
english2 = [["the","dog"], ["the","house"], ["the", "green", "house"]]
polish = [["pies"], ["dom"], ["zielony", "dom"]]

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Sprawdź jak wyglądają prawdopodobieństwa tłumaczeń po 10 iteracjach.

In [ ]:
translation_prob

Sprawdź czy gdybyś dodał słówko `NULL` to algorytm nauczyłby się wiązać słówko `NULL` na `the`, które nie występuje w języku polskim?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
translation_prob

Jeśli wywołałbyś EM dla pierwszego korpusu równoległego (zmienne `english` i `spanish`) i dołączył tokeny `NULL` to EM tłumaczy NULL jako "casa" i "house" jako "casa" z takimi samymi prawdopodobieństwami. Dlaczego?

YOUR ANSWER HERE